In [ ]:
from mlrun import NewTask, get_run_db, new_function, run_local
import os
from mlrun.utils import add_vault_project_secret, add_vault_user_secret
from mlrun import mlconf, code_to_function, new_project

In [ ]:
proj_name = 'vault-mlrun'

proj = new_project(proj_name, init_vault=True)
proj.create_vault_secrets({'aws_key': '1234567890', 'github_key': 'proj1Key!!!'})

In [ ]:
func_path = os.path.abspath("vault_function.py")

func = code_to_function(filename=func_path,
                        handler='vault_func',
                        project=proj,
                        kind='job')

In [ ]:
func.spec.build.base_image = 'saarcoiguazio/mlrun:unstable'
func.spec.build.image = '.secret-image'
func.deploy()

In [ ]:
task = NewTask(project=proj_name,
               name='vault_test_run',
               handler='vault_func',
               params={'secrets':['password', 'path', 'github_key', 'aws_key']})
task.with_secrets('vault', {"project": proj_name, "secrets": ["aws_key", "github_key"]})


In [ ]:
result = func.run(task)